In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SeparableConv2D, BatchNormalization, MaxPooling2D
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Activation

# Conv Block A
def conv_block_a(filters, input_shape=None):
    layers = [
        SeparableConv2D(filters, kernel_size=3, strides=3, activation='relu', padding='same', input_shape=input_shape),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2))
    ]
    block = Sequential(layers)
    return block

# Conv Block B
def conv_block_b(filters):
    block = Sequential([
        SeparableConv2D(filters, kernel_size=3, strides=3, activation='relu', padding='same'),
        BatchNormalization()
    ])
    return block

# Conv Block C
def conv_block_c():
    block = Sequential([
        SeparableConv2D(512, kernel_size=3, strides=3, activation='relu', padding='same'),
        GlobalAveragePooling2D(),
        Dense(256, activation='relu'),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(2, activation='sigmoid')
    ])
    return block

/software/spackages/linux-rocky8-x86_64/gcc-9.5.0/anaconda3-2022.05-zyrazrj6uvrtukupqzhaslr63w7hj6in/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

# Load your data
# Load your data
X = np.load('wavelet_transformed_data/layered_452x452_normalized.npy')

# Load labels and convert them to integers
labels_df = pd.read_csv('labels/452x452.csv')
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(labels_df['Label'].values)
y = to_categorical(integer_encoded)

# Define input shape based on X
input_shape = X.shape[1:]

# Extract height, width, and channels from X.shape
#height, width, channels = X.shape[1], X.shape[2], X.shape[3]

# Define your input shape
#input_shape = (height, width, channels)

# Building the CNN
model = Sequential()

# Adding Conv Block A with input shape
model.add(conv_block_a(256, input_shape=input_shape))

# Adding 5 Conv Block B
for _ in range(5):
    model.add(conv_block_b(256))

# Adding Conv Block C
model.add(conv_block_c())

# Model Summary
model.summary()

# Compiling the Model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_val, y_val))


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_1 (Sequential)   (None, 38, 38, 256)       2075      
                                                                 
 sequential_2 (Sequential)   (None, 13, 13, 256)       69120     
                                                                 
 sequential_3 (Sequential)   (None, 5, 5, 256)         69120     
                                                                 
 sequential_4 (Sequential)   (None, 2, 2, 256)         69120     
                                                                 
 sequential_5 (Sequential)   (None, 1, 1, 256)         69120     
                                                                 
 sequential_6 (Sequential)   (None, 1, 1, 256)         69120     
                                                                 
 sequential_7 (Sequential)   (None, 2)                 2

In [5]:
import os

# Define the directory to save the model
model_dir = 'models'

# Create the directory if it doesn't exist
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

# Save the model
model_save_path = 'models/452x452_tow-ids_model.h5'

# Save the model in HDF5 format
model.save(model_save_path, save_format='h5')

print(f"Model saved to {model_save_path}")

Model saved to models/452x452_tow-ids_model.h5


/home/lnagana/.local/lib/python3.9/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [16]:
# Reshape sample_data to add batch dimension
sample_data = test_data[90] 
sample_data = np.expand_dims(sample_data, axis=0)  # Reshape to (1, height, width, channels)
sample_label = test_labels[90]

# Make prediction
sample_prediction = model.predict(sample_data)

# Convert predictions to class labels
predicted_class = np.argmax(sample_prediction, axis=1)
true_class = np.argmax(sample_label)  # No need to use axis=1 here as it's a single label

# label_encoder is a fitted LabelEncoder instance
predicted_class_label = label_encoder.inverse_transform(predicted_class)[0]
true_class_label = label_encoder.inverse_transform([true_class])[0]

print(f"Predicted Class: {predicted_class_label}")
print(f"True Class: {true_class_label}\n")


(1, 228, 228, 3)
1/1 [==============================] - 0s 25ms/step
Predicted Class: Normal
True Class: Normal

